In [1]:
import requests 
from requests_ntlm import HttpNtlmAuth
from subprocess import call

from telebot import types
import telebot 
import os


import datetime
import pyodbc
pyodbc.pooling=False

In [2]:
import numpy as np 
import pandas as pd 
from glob import glob
import cv2

from tensorflow import keras
import keras
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, 
                          Dropout, Rescaling, RandomFlip, RandomRotation, BatchNormalization)

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers

C:\Users\a.dubinets\opencv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\a.dubinets\opencv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\a.dubinets\opencv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
token ='1670417933:AAGL36aYrdY2slu_GKrya83oI_dTYCx5gUU'   
URL='https://api.telegram.org/bot1670417933:AAGL36aYrdY2slu_GKrya83oI_dTYCx5gUU/getMe'
im_path=r'C:\Users\a.dubinets\Desktop\Обучение\Mushrooms\Dataset\test_image'
all_mushrooms=pd.read_excel(r'C:\Users\a.dubinets\Desktop\Обучение\Mushrooms\Result\mushroom.xlsx') 
class_train =  pd.read_csv(r'C:\Users\a.dubinets\Desktop\Обучение\Mushrooms\Result\class.csv', header=None)
_class = class_train.iloc [:, 0].tolist()
os.chdir(im_path)

In [4]:
" ".join(all_mushrooms['title'].head(1).values.tolist()) 

'Свинушка тонкая'

In [5]:
# загрузим нашу модель 
model = load_model(r'C:\Users\a.dubinets\Desktop\Обучение\Диплом\mushrooms-ResNet50.hdf5')

In [6]:
# загружаем входное изображение и предобрабатываем
IMG_SIZE = (240, 240)
def load_image(path, target_size=IMG_SIZE):   # загружает картинку и делает ресайз
    imageBGR = cv2.imdecode(np.fromfile(path, dtype=np.uint8), cv2.IMREAD_COLOR)
    imageRGB = cv2.cvtColor(imageBGR , cv2.COLOR_BGR2RGB)
    img = cv2.resize(imageRGB, (240,240))
    return  img  # предобработка 
 

# функция-генератор загрузки тестовых изображений с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])
            
def pred(path):
    im_path=r'C:\Users\a.dubinets\Desktop\Обучение\Mushrooms\Dataset\test_image'
    os.chdir(im_path)
    test_files=glob(path)
    test_pred = model.predict(predict_generator(test_files), steps=len(test_files))   
#    print(len(test_pred[0]),len(_class))
    intermediate_dictionary = {'proba':test_pred[0], 'title':_class}
    df = pd.DataFrame(intermediate_dictionary)
    df = df.sort_values(by='proba', ascending=False)
    df = df.head(10)
#    df_fix = pd.merge(df, all_mushrooms, left_on='title')
    res = " ".join(df['title'].head(1).values.tolist()) 
    return df, res

In [22]:
mushroom_list=pd.read_excel(r'C:/Users/a.dubinets/Desktop/Обучение/Mushrooms/Result/mushroom.xlsx') 
mushroom_list['title']= "Гриб_"  + df['title']
mushroom_list['title'].replace('\s+', '_',regex=True,inplace=True)
mushroom_list['desciption'].replace('\n', '',regex=True,inplace=True)
mushroom_list.head(5)

,title,lat,link,desciption
0,Гриб_Свинушка_тонкая,Paxillus involutus,https://wikigrib.ru/svinushka-tonkaya/,Свинушка тонкая (лат. Paxillus involutus) или ...
1,Гриб_Жёлчный_гриб,Tylopilus felleus,https://wikigrib.ru/zhelchnyj-gribgorchak/,Воплощение разочарования любого грибника – Лож...
2,Гриб_Рядовка_лиловоногая,Lepista saeva,https://wikigrib.ru/ryadovka-lilovonogaya/,Характеристики:Группа: пластинчатыеПластинки: ...
3,Гриб_Рядовка_тополёвая,Tricholoma populinum,https://wikigrib.ru/ryadovka-topolevaya/,Характеристики:Группа: пластинчатыеЦвет: красн...
4,Гриб_Опёнок_осенний,Armillaria mellea; Armillaria borealis,https://wikigrib.ru/openok-osennij/,Характеристики:Группа: пластинчатыеПластинки: ...


In [ ]:
#Тело бота 
bot = telebot.TeleBot(token)
# ---------------------------------------------
@bot.message_handler(content_types=['text'])
def start(message):  
#   Блок 1. Обновление основных данных для отчета
    if  message.text == 'Привет':   # Получаем основные отчеты.   
        bot.send_message(message.from_user.id, "О, привет)")  
    elif message.text == '/start':    
        bot.send_message(message.from_user.id,'"Данный бот является дипломной работой на курсе изучения алгоритмов dl. Бот предназначен для распознавания наиболее вероятного названия гриба, а также предоставление дополнительной информации о нем в качестве ссылки на сайт https://wikigrib.ru/. Бот создан в развлекательных целях и автор не несет ответственности за принятые решения на основе данного приложения. Помните, что определить является ли гриб съедобным или нет, может эксперт с многолетним опытом. Данная программа предназначена лишь в качестве помощи по нахождению описания того или иного гриба, а также для изучения удивительного мира грибов и информации о них.  Для использования бота, вам нужно выслать фотографию гриба. \n 1) Фотография должна быть четкой. \n 2) На 1 фотографии должен быть 1 гриб, либо грибы одного вида. \n 3) Вы можете сфотографировать 2 гриба одного вида, один из которых будет в профиль, а другой фас. Это должно улучшить качество. \n \n Удачной тихой охоты. =)))."')
    else:
        bot.send_message(message.from_user.id, 'Для распознавания вышлите фото гриба. Для информации нажмите /start')     
@bot.message_handler(content_types=['photo'])
def photo(message):
    file_info = bot.get_file(message.photo[len(message.photo)-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    src=file_info.file_path;
    print(src)
    with open(src, 'wb') as new_file:
        new_file.write(downloaded_file)        
    bot.reply_to(message,"Анализирую фото...")
    df, res = pred(src)
    
    bot.reply_to(message,"Я думаю что это: "+res)
    
    # модуль по анализу вероятности
    result = pd.merge(df, mushroom_list, on='title', how='left')
    result.to_html('Mushroom_proba.html',index=True, encoding='windows-1251')      
    t=open(r'Mushroom_proba.html', 'rb')
    bot.send_document(message.from_user.id, t)
    
bot.infinity_polling(1) #True

photos/file_14.jpg
1/1 [==============================] - 0s 33ms/step
photos/file_15.jpg
1/1 [==============================] - 0s 41ms/step
photos/file_16.jpg
1/1 [==============================] - 0s 30ms/step
photos/file_17.jpg
1/1 [==============================] - 0s 29ms/step
photos/file_18.jpg
1/1 [==============================] - 0s 31ms/step
photos/file_19.jpg
1/1 [==============================] - 0s 37ms/step
photos/file_20.jpg
1/1 [==============================] - 0s 30ms/step
photos/file_21.jpg
1/1 [==============================] - 0s 28ms/step
